<a href="https://colab.research.google.com/github/david7franco/david7franco/blob/main/Copy_of_NB_2_DataFrame_and_Series_Manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataframe Creation and Manipulation Using Pandas
Pandas is an excellent library of python for data anlysis. Pandas uses NumPy internally but sometimes we will need to access its functions directly so we can import that also.

<a href=https://pandas.pydata.org/docs/reference/index.html#api> Pandas Documentation </a>

<a href=https://numpy.org/doc/stable/reference/index.html#reference> NumPy Documentation </a>

In [ ]:
import numpy as np
import pandas as pd

## Reading Data into a DataFrame Object in Pandas

Pandas allows us to read in (and write out) data from many file types both locally and on a network.
- CSV
- SQL
- JSON
- HTML
- XML
- Excel
- Etc.

In [ ]:
# Let's read in a CSV file from GetHub.
url = 'https://raw.githubusercontent.com/ine-rmotr-curriculum/FreeCodeCamp-Pandas-Real-Life-Example/master/data/sales_data.csv'
sales = pd.read_csv(url)
sales.head(3)

## Types of Data

### Statistical Data Types
* Categorical vs Numerical
    * Discrete vs Continuous
* Levels of Measurement
    * Nominal
    * Ordinal
    * Interval
    * Ratio

### Python Data Types
* Strings
* Sets
* Dictionaries
* Booleans
* None
* Sequences
    * Tuples
    * Lists
    * Ranges
* Numbers
    * Integers
    * Floats
    * Complex

In [ ]:
# We can use the info method to see a summary of how Pandas is storing our data.
sales.info()

## Cleaning Up Our DataFrame
If we wanted to otimize this data for processing speed we should minimize the use of objects.  
(Those are not processed any faster by NumPy than they are in basic Python.)
- Pandas has a datetime dtype <font color="red"> and we could use the astype method to parse the Date column. </font>
- The day, month, and year columns are redudant, so we could delete them using the drop method.   
    (We can use the datetime data type to pull those values if we want them.)
- <font color="red"> We can use the category dtype to make the storage and processing of some string columns more efficient.  
    (We can even create <a href=https://pandas.pydata.org/docs/reference/api/pandas.CategoricalDtype.html> custom category dtypes </a> that give our categories order.) </font>  
- <font color="red"> We can reduce the size of our integer variables to save space and processing time. </font>


In [ ]:
# Use astype method to 
sales = sales.astype({'Date':'datetime64'})
# Use the drop method to remove columns from a dataframe
sales.drop(labels=['Day', 'Month', 'Year'], axis=1, inplace=True)
# Pass the object columns into astype to reformat them as category data.
sales = sales.astype({'Age_Group':'category', 
                      'Customer_Gender':'category', 
                      'Country':'category', 
                      'State':'category', 
                      'Product_Category':'category', 
                      'Sub_Category':'category', 
                      'Product':'category'})
# Pass the integer columns into astype to reduce size where possible.
sales = sales.astype({'Customer_Age':'int8', 
                      'Order_Quantity':'int8', 
                      'Unit_Cost':'int16', 
                      'Unit_Price':'int16', 
                      'Profit':'int16', 
                      'Cost':'int16', 
                      'Revenue':'int16'})
sales.info()

Here is a whole list of methods you can use to get info from the datetime data type in Pandas.  
<a href=https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html> Timestamp Methods </a>

In [ ]:
# Lets get back the month name for each row using the apply and the month_name methods.
sales['Date'].apply(pd.Timestamp.month_name)

### You Try It: Timestamp Methods
In the code blocks below:  
- Determine the name of the day for each of the entries in our data set.
- Determine the numerical day of the year for each of the entries in our data set.  
<font color="green"> Hint: Remember that the input for the apply method must be a function. </font>
- Determine if that date fell on a leap year for each of the entries in our data set.  
<font color="green"> Hint: Remember that the input for the apply method must be a function. </font>

In [ ]:
sales['Date'].apply(pd.Timestamp.day_name)

In [ ]:
sales['Date'].apply(lambda x: x.day_of_year)

In [ ]:
sales['Date'].apply(lambda x: x.is_leap_year)

## Useful DataFrame Values and Methods
- head/tail
- info
- index
- columns
- dtypes
- <font color='red'> astype </font>
- values
- items
- shape
- apply
- drop/pop
- dropna/fillna
- copy

## Accessing Subsets of a DataFrame
We can use the standard Python indexing to call individual or groups of columns by name.  
An individual colum will return a different Pandas object called a Series.

In [ ]:
# Let's just look at the Customer_Age column
sales['Customer_Age']

In [ ]:
# How about Customer_Age and Customer_Gender together
sales[['Customer_Age', 'Customer_Gender']]

We can access individual or groups of rows by numerical index using iloc or by labels or boolean array using loc.

In [ ]:
# Access just the first row
sales.iloc[0]

In [ ]:
# Access the first 1000 rows
sales.iloc[:1000]

**Creating** a boolean array (mask) for data you want is quite easy in Pandas.

In [ ]:
# Let's say we want the ages of all sales in Germany.
mask = sales['Country'] == 'Germany'
mask

In [ ]:
germany_rows = sales.loc[mask]
germany_rows

In [ ]:
germany_rows['Customer_Age']

In [ ]:
# Let's try a harder one like the age groups of all the canadians who bought vests
mask = (sales['Country'] == 'Canada') & (sales['Sub_Category'] == 'Vests')
sales.loc[mask, 'Age_Group']

### You Try It: Subsets of the Data
- Find the gender of customers who live in France
- Find the age range of customers who spent more than $1000 and didn't buy vests.
- Find the age of customers who don't live in Germany and ordered in 2015.



In [ ]:
mask = (sales['Country'] == 'France')
sales.loc[mask,'Customer_Gender']

In [ ]:
age_range = (sales['Sub_Category'] != 'Vests') & (sales['Revenue'] > 1000)
sales.loc[age_range] 

In [ ]:
germ = (sales['Country'] != 'Germany') & (sales['Date'].apply(lambda x: x.year) == 2015)
sales.loc[germ, 'Customer_Age']

## Applying Functions to the Data
- Apply Method 
- Vectorized Functions

Let's define a function we want to apply to each row of our data.  
We can encode the Customer_Gender variable as M=0 and F=1

In [ ]:
# First we make sure those are our only options in the dataframe
sales['Customer_Gender'].value_counts()

In [ ]:
# Let's use the apply method first
def encode_gender(row):
    if row['Customer_Gender'] == 'M':
        return 0
    else: return 1

def decode_gender(row):
    if row['Cutomer_Gender'] == 0:
        return 'M'
    else: return 'F'

In [ ]:
# Use the apply method to itterate though the rows or columns of the dataframe and apply the function
sales.apply(encode_gender, axis=1)

In [ ]:
# Lets see how long on average the apply version takes to run.
%%timeit
sales_encoded = sales.copy()
sales_encoded['Customer_Gender'] = sales.apply(encode_gender, axis=1)

In [ ]:
# How long does it take on average for the vectorized version of this?
%%timeit
sales_encoded = sales.copy()
sales_encoded['Customer_Gender'] = 0 
mask = sales['Customer_Gender'] == 'F'
sales_encoded.loc[mask, 'Customer_Gender'] = 1

In [ ]:
# Let's make a new dataframe with our encoded column
sales_encoded = sales.copy()
sales_encoded['Customer_Gender'] = 0 
mask = sales['Customer_Gender'] == 'F'
sales_encoded.loc[mask, 'Customer_Gender'] = 1
sales_encoded.info()

### You Try It: Applying Functions
Use the apply function or vectorized functions to do the following:
- Calculate the profit per unit for each order.
- Encode the Age_Group column into interger values.
- Replace the Age_Group column with the encoded values in the encoded dataframe.

In [ ]:
def profit_per_unit(row):
    return row['Profit']/ row['Order_Quatity']

    sales.apply(profit_per_unit, axis =1)

In [ ]:
%timeit sales['Profit']/sales['Order_Quantity']

## Createing New Columns
It is very easy to create a new column at the end using the indexing by label.  
You can also use the insert method to place the column elsewhere.

In [ ]:
# Lets make a new nonsensical column of age*profit.
sales['Age-Profit'] = sales['Customer_Age'] * sales['Profit']

In [ ]:
sales.head()

In [ ]:
sales.drop(labels=['Age-Profit'], axis=1, inplace=True)
sales.insert(1, 'Age_Profit', sales['Customer_Age'] * sales['Profit'])
sales.head()

### You Try It: New Columns and Rows
- Drop the obsurd Age-Profit column that some clown added.
- Add a new column for the profit per unit for each order.

## Where to Find Datasets
Here are a few links to sites with many sources:  

<a href=https://libguides.rowan.edu/data> Rowan Library Guide to Data Sources </a>  
<a href=https://www.kaggle.com/> Kaggle </a>  
<a href=https://datasetsearch.research.google.com/> Google Dataset Search </a>

There are many specialized portals for specific types of data. For example,  
<a href=https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/home> FBI: Crime Data Explorer </a>  
<a href=https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page> NYC: Taxi Trip Data </a>  
<a href=https://apps.who.int/gho/data/node.home> WHO: Global Heath Observatory Data Repository </a>


## Ways to Save DataFrames to File
- CSV (good for sharing with non-pandas users)
- Pickle (standard python serialization)
- Parquet (good for long term storage)
- Feather (good for short term storage)

In [ ]:
%timeit sales.to_csv('/content/sales_csv')
%timeit pd.read_csv('/content/sales_csv')
!ls -lh 'sales_csv'
pd.read_csv('/content/sales_csv').info()

In [ ]:
%timeit sales.to_pickle('/content/sales_pickle')
%timeit pd.read_pickle('/content/sales_pickle')
!ls -lh 'sales_pickle'
pd.read_pickle('/content/sales_pickle').info()

In [ ]:
%timeit sales.to_parquet('/content/sales_parquet')
%timeit pd.read_parquet('/content/sales_parquet')
!ls -lh 'sales_parquet'
pd.read_parquet('/content/sales_parquet').info()

In [ ]:
%timeit sales.to_feather('/content/sales_feather')
%timeit pd.read_feather('/content/sales_feather')
!ls -lh 'sales_feather'
pd.read_feather('/content/sales_feather').info()